In [1]:
import os
import json
import datetime
import warnings
import polars as pl
import pandas as pd
import altair as alt

from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.hezke_jmeno import hezke_jmeno
from src.kristi_promin import kristi_promin
from src.zjisti_vazbu import zjisti_vazbu
from src.me_to_neurazi import me_to_neurazi

pl.Config(tbl_rows=100)
alt.data_transformers.disable_max_rows()
alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

with open(os.path.join('src','kredity.json'), 'r', encoding='utf-8') as kredity:
    kredity = json.loads(kredity.read())

In [3]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bez_bordelu, return_dtype=str))
df = df.explode("020_q").with_columns(pl.col("020_q").map_elements(zjisti_vazbu, return_dtype=str).alias('vazba'))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))
print(len(df))
df = df.filter(~pl.col('rok').is_null()).sort(by='rok')
df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null())
print(len(df))

1001279
991612


In [4]:
df = df.filter(pl.col("rok") >= 1800)

In [5]:
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","041.parquet")), left_on="001", right_on="001", how="left")

In [6]:
predfiltr = df.filter(pl.col('stran') >= 30).unique(subset=['rok','245_a','100_a'])

In [7]:
preklady = predfiltr.drop_nulls(subset=["041_h","rok"]).explode("041_h").filter(pl.col("041_h").is_not_null()).filter(pl.col("041_h") != "cze").unique(subset=['rok','245_a','100_a'])

In [15]:
do_grafu = preklady.group_by("rok").agg(pl.col("041_h").unique().len())

In [57]:
pocty_jazyku = alt.Chart(
    alt_friendly(do_grafu.filter(pl.col('rok').is_between(1850,2024))),
    title=["Z kolika různých cizích jazyků vyšly knižní překlady"],
    width=kredity['sirka'],
    height=kredity['vyska_nizkych']
).mark_bar(width=1.5, color='#70871E').encode(
    alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')),
    alt.Y("041_h:Q", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6', orient='right'))
).configure_view(
    stroke='transparent'
).configure_axisX(
    grid=False, domain=False
)

pocty_jazyku

alt.Chart(...)

In [59]:
me_to_neurazi(pocty_jazyku, soubor='04_pocty_jazyku', kredity=kredity['default'])

<figure>
    <a href="https://data.irozhlas.cz/knihy-grafy/04_pocty_jazyku.svg" target="_blank">
    <img src="https://data.irozhlas.cz/knihy-grafy/04_pocty_jazyku.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>
